In [6]:
import random

# Poker Monte Carlo Simulator

This project uses Monte Carlo simulation to estimate the probability of winning a game of Texas Hold’em with a given a user inputted hand. It models 1M+ randomized trials per input using simulated community cards and random opponent hands, allowing users to assess expected performance and strategic value under uncertainty.

In [3]:
# Functions that simulate dealing cards in a poker game
def create_deck():
    """Create a standard 52-card deck"""
    ranks = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']
    suits = ['H', 'D', 'C', 'S']
    return [rank + suit for rank in ranks for suit in suits]

def shuffle_deck(deck):
    """Shuffle the deck"""
    random.shuffle(deck)
    return deck

def draw_card(deck):
    """Draw a card from the deck"""
    return deck.pop()

def sim_flop(deck):
    """Simulate dealing the flop (3 community cards)"""
    return [draw_card(deck) for _ in range(3)]

def sim_turn(deck):
    """Simulate dealing the turn (1 community card)"""
    return draw_card(deck)

def sim_river(deck):
    """Simulate dealing the river (1 community card)"""
    return draw_card(deck)



In [ ]:
# Initialize counts
royal_flush_count = 0
straight_flush_count = 0
four_of_a_kind_count = 0
full_house_count = 0
flush_count = 0
straight_count = 0
three_of_a_kind_count = 0
two_pair_count = 0
one_pair_count = 0
high_card_count = 0

def card_value(card):
    """Convert card rank to numerical value for comparison"""
    rank = card[:-1]  # Remove suit
    if rank == 'A':
        return 14
    elif rank == 'K':
        return 13
    elif rank == 'Q':
        return 12
    elif rank == 'J':
        return 11
    else:
        return int(rank)

def get_suit(card):
    """Get the suit of a card"""
    return card[-1]

def is_flush(cards):
    """Check if cards form a flush"""
    suits = [get_suit(card) for card in cards]
    for suit in ['H', 'D', 'C', 'S']:
        if suits.count(suit) >= 5:
            kicker = max(card_value(card) for card in cards if get_suit(card) == suit)
            return True, suit, kicker
    return False, None, 0

def is_straight(values):
    """Check if values form a straight"""
    unique_values = sorted(set(values))
    
    # Check for A-2-3-4-5 straight (wheel)
    if set([14, 2, 3, 4, 5]).issubset(set(unique_values)):
        return True, 5
    
    # Check for regular straights
    for i in range(len(unique_values) - 4):
        if unique_values[i+4] - unique_values[i] == 4:
            return True, unique_values[i+4]
    return False, 0

def hand_strength(seven_cards):
    """Evaluate the strength of a 7-card hand and return (rank, high_cards)"""

    # Count occurrences of each card value
    values = [card_value(card) for card in seven_cards]
    value_counts = {}
    for value in values:
        value_counts[value] = value_counts.get(value, 0) + 1
    
    # Puts counts in order of strength (card value, count)
    sorted_counts = sorted(value_counts.items(), key=lambda x: (x[1], x[0]), reverse=True)
    
    is_flush_result, flush_suit, flush_kicker = is_flush(seven_cards)
    is_straight_result, straight_high = is_straight(values)
    
    # Straight Flush / Royal Flush
    if is_flush_result and is_straight_result:
        flush_cards = [card for card in seven_cards if get_suit(card) == flush_suit]
        flush_values = [card_value(card) for card in flush_cards]
        is_straight_flush, sf_high = is_straight(flush_values)
        if is_straight_flush:
            if sf_high == 14:  # Royal flush
                royal_flush_count += 1
                return (9, [14])
            else:  # Straight flush
                straight_flush_count += 1
                return (8, [sf_high])
    
    # Four of a kind
    if sorted_counts[0][1] == 4:
        four_of_a_kind_count += 1
        return (7, [sorted_counts[0][0], sorted_counts[1][0]])
    
    # Full house
    elif sorted_counts[0][1] == 3 and sorted_counts[1][1] >= 2:
        full_house_count += 1
        return (6, [sorted_counts[0][0], sorted_counts[1][0]])
    
    # Flush
    elif is_flush_result:
        flush_count += 1
        return (5, [flush_kicker])
    
    # Straight
    elif is_straight_result:
        straight_count += 1
        return (4, [straight_high])
    
    # Three of a kind
    elif sorted_counts[0][1] == 3:
        three_of_a_kind_count += 1
        kickers = [item[0] for item in sorted_counts[1:3]]
        return (3, [sorted_counts[0][0]] + kickers)
    
    # Two pair
    elif sorted_counts[0][1] == 2 and sorted_counts[1][1] == 2:
        two_pair_count += 1
        pairs = [sorted_counts[0][0], sorted_counts[1][0]]
        kicker = sorted_counts[2][0]
        return (2, pairs + [kicker])
    
    # One pair
    elif sorted_counts[0][1] == 2:
        one_pair_count += 1
        pair = sorted_counts[0][0]
        kickers = [item[0] for item in sorted_counts[1:4]]
        return (1, [pair] + kickers)
    
    # High card
    else:
        high_card_count += 1
        high_cards = [item[0] for item in sorted_counts[:5]]
        return (0, high_cards)
    
    

def compare_hands(hand1_strength, hand2_strength):
    """Compare two hands. Returns 1 if hand1 wins, -1 if hand2 wins, 0 if tie"""
    rank1, values1 = hand1_strength
    rank2, values2 = hand2_strength
    
    if rank1 > rank2:
        return 1
    elif rank2 > rank1:
        return -1
    else:
        # Same rank, compare high cards
        for v1, v2 in zip(values1, values2):
            if v1 > v2:
                return 1
            elif v2 > v1:
                return -1
        return 0  # Tie

def simulate_games(player1_cards, player2_cards, remaining_deck, flop_cards=None, turn_card=None, river_card=None, num_simulations=1000000):
    """Simulate poker games and return win/tie/loss probabilities for player 1"""
    player1_wins = 0
    ties = 0
    player2_wins = 0
    
    for _ in range(num_simulations):
        # Make a copy of the remaining deck for this simulation
        sim_deck = remaining_deck.copy()
        random.shuffle(sim_deck)

        # Deal community cards if not provided, and remove them from the deck on each iteration
        if flop_cards is None:
            sim_flop = [draw_card(sim_deck) for _ in range(3)]
        else:
            sim_flop = flop_cards.copy()
        
        if turn_card is None:
            sim_turn = draw_card(sim_deck)
        else:
            sim_turn = turn_card
        
        if river_card is None:
            sim_river = draw_card(sim_deck)
        else:
            sim_river = river_card
        
        # Create 7-card hands for both players
        community_cards = sim_flop + [sim_turn, sim_river]
        player1_seven = player1_cards + community_cards
        player2_seven = player2_cards + community_cards
        
        # Evaluate hand strengths
        player1_strength = hand_strength(player1_seven)
        player2_strength = hand_strength(player2_seven)
        
        # Compare hands
        result = compare_hands(player1_strength, player2_strength)
        
        if result == 1:
            player1_wins += 1
        elif result == -1:
            player2_wins += 1
        else:
            ties += 1
    
    # Calculate probabilities
    win_prob = player1_wins / num_simulations
    tie_prob = ties / num_simulations
    loss_prob = player2_wins / num_simulations
    
    print(f"\nResults after {num_simulations:,} simulations:")
    print(f"Player 1 Win Probability:  {win_prob:.4f} ({win_prob*100:.2f}%)")
    print(f"Tie Probability:          {tie_prob:.4f} ({tie_prob*100:.2f}%)")
    print(f"Player 1 Loss Probability: {loss_prob:.4f} ({loss_prob*100:.2f}%)")
    
    return win_prob, tie_prob, loss_prob

def play_game():
    deck = create_deck()
    remaining_deck = shuffle_deck(deck)

    # Input player and opponent cards
    print("Enter your two cards (e.g. 'AH' for Ace of Hearts):")
    player1_card1 = input("Card 1: ").strip().upper()
    player1_card2 = input("Card 2: ").strip().upper()

    print("Enter your suspected opponent's two cards (e.g. 'AH' for Ace of Hearts):")
    player2_card1 = input("Card 1: ").strip().upper()
    player2_card2 = input("Card 2: ").strip().upper()

    # Create player card hands (as lists)
    player1_cards = [player1_card1, player1_card2]
    player2_cards = [player2_card1, player2_card2]

    # Remove player cards from deck
    for card in player1_cards + player2_cards:
        remaining_deck.remove(card)

    # Ask for current round
    print("What round is it? (preflop = P, flop = F, turn = T, river = R):")
    round_input = input("Round: ").strip().upper()

    if round_input == 'P':
        simulate_games(player1_cards, player2_cards, remaining_deck, num_simulations=1000000)
    elif round_input == 'F':
        print("Enter the flop cards:")
        flop1 = input("Flop Card 1: ").strip().upper()
        flop2 = input("Flop Card 2: ").strip().upper()
        flop3 = input("Flop Card 3: ").strip().upper()
        flop_cards = [flop1, flop2, flop3]
        for card in flop_cards:
            remaining_deck.remove(card)
        simulate_games(player1_cards, player2_cards, remaining_deck, flop_cards=flop_cards, num_simulations=1000000)
    elif round_input == 'T':
        print("Enter the flop cards:")
        flop1 = input("Flop Card 1: ").strip().upper()
        flop2 = input("Flop Card 2: ").strip().upper()
        flop3 = input("Flop Card 3: ").strip().upper()
        print("Enter the turn card:")
        turn_card = input("Turn Card: ").strip().upper()
        flop_cards = [flop1, flop2, flop3]
        for card in flop_cards + [turn_card]:
            remaining_deck.remove(card)
        simulate_games(player1_cards, player2_cards, remaining_deck, flop_cards=flop_cards, turn_card=turn_card, num_simulations=1000000)
    elif round_input == 'R':
        print("Enter the flop cards:")
        flop1 = input("Flop Card 1: ").strip().upper()
        flop2 = input("Flop Card 2: ").strip().upper()
        flop3 = input("Flop Card 3: ").strip().upper()
        print("Enter the turn card:")
        turn_card = input("Turn Card: ").strip().upper()
        print("Enter the river card:")
        river_card = input("River Card: ").strip().upper()
        flop_cards = [flop1, flop2, flop3]
        for card in flop_cards + [turn_card, river_card]:
            remaining_deck.remove(card)
        simulate_games(player1_cards, player2_cards, remaining_deck, flop_cards=flop_cards, turn_card=turn_card, river_card=river_card, num_simulations=1000000)


## Testing the Simulator

Let's test the simulator with some example hands to verify it works correctly.

In [35]:
# Test the hand strength evaluation with some example hands

# Test 1: Royal Flush vs Full House
royal_flush = ['AH', 'KH', 'QH', 'JH', '10H', '2C', '3D']
full_house = ['AH', 'AC', 'AD', 'KH', 'KC', '2C', '3D']

rf_strength = hand_strength(royal_flush)
fh_strength = hand_strength(full_house)

print("Royal Flush strength:", rf_strength)
print("Full House strength:", fh_strength)
print("Winner:", "Royal Flush" if compare_hands(rf_strength, fh_strength) == 1 else "Full House")
print()

# Test 2: Quick simulation with known strong vs weak hands
print("Testing AA vs 72 (strongest vs weakest starting hands):")
strong_hand = ['AH', 'AS']
weak_hand = ['7C', '2D']

# Create a deck without these cards
test_deck = create_deck()
for card in strong_hand + weak_hand:
    test_deck.remove(card)

# Run a smaller simulation for testing
win_prob, tie_prob, loss_prob = simulate_games(strong_hand, weak_hand, test_deck, num_simulations=10000)
print(f"Expected: AA should win ~85% of the time")
print()

Royal Flush strength: (9, [14])
Full House strength: (6, [14, 13])
Winner: Royal Flush

Testing AA vs 72 (strongest vs weakest starting hands):

Results after 10,000 simulations:
Player 1 Win Probability:  0.8697 (86.97%)
Tie Probability:          0.0043 (0.43%)
Player 1 Loss Probability: 0.1260 (12.60%)
Expected: AA should win ~85% of the time



In [ ]:
play_game()

## How to Use the Simulator

### Interactive
To run the poker simulator, run the `play_game()` function above. It will prompt you for:

1. **Your two cards in hand** (e.g., 'AH' for Ace of Hearts)
2. **Opponent's suspected cards in hand**
3. **Current round**: 
   - P = Preflop
   - F = Flop (enter the 3 flop cards)
   - T = Turn (enter flop + turn cards)
   - R = River (enter all community cards e.g: flop + turn + river)

### Card Format
- Ranks: 2, 3, 4, 5, 6, 7, 8, 9, 10, J, Q, K, A
- Suits: H (Hearts), D (Diamonds), C (Clubs), S (Spades)
- Examples: AH, KS, 10D, 2C

### Hand Rankings (from strongest to weakest)
1. Royal Flush (A-K-Q-J-10 all same suit)
2. Straight Flush (5 consecutive cards, same suit)
3. Four of a Kind
4. Full House (3 of a kind + 1 pair)
5. Flush (5 cards same suit)
6. Straight (5 consecutive cards)
7. Three of a Kind
8. Two Pair
9. One Pair
10. High Card

In [37]:
# Example: Programmatic usage without user input
# Calculate probability of pocket aces vs pocket kings preflop

def quick_simulation(hand1, hand2, num_sims=100000):
    """Quick function to simulate two specific hands"""
    deck = create_deck()
    for card in hand1 + hand2:
        deck.remove(card)
    
    return simulate_games(hand1, hand2, deck, num_simulations=num_sims)

# Example calculations
print("=== Famous Poker Hand Matchups ===")
print()

# AA vs KK (classic preflop confrontation)
print("1. Pocket Aces vs Pocket Kings:")
win, tie, loss = quick_simulation(['AH', 'AS'], ['KH', 'KS'], 50000)
print()

# AK vs QQ (coin flip situation)
print("2. Ace-King vs Pocket Queens (classic coin flip):")
win, tie, loss = quick_simulation(['AH', 'KS'], ['QH', 'QS'], 50000)
print()

# 72o vs random (worst starting hand)
print("3. 7-2 offsuit vs Ace-King suited:")
win, tie, loss = quick_simulation(['7C', '2D'], ['AH', 'KH'], 50000)
print()

=== Famous Poker Hand Matchups ===

1. Pocket Aces vs Pocket Kings:

Results after 50,000 simulations:
Player 1 Win Probability:  0.8243 (82.43%)
Tie Probability:          0.0054 (0.54%)
Player 1 Loss Probability: 0.1704 (17.04%)

2. Ace-King vs Pocket Queens (classic coin flip):

Results after 50,000 simulations:
Player 1 Win Probability:  0.4303 (43.03%)
Tie Probability:          0.0081 (0.81%)
Player 1 Loss Probability: 0.5616 (56.16%)

3. 7-2 offsuit vs Ace-King suited:

Results after 50,000 simulations:
Player 1 Win Probability:  0.3035 (30.35%)
Tie Probability:          0.0055 (0.55%)
Player 1 Loss Probability: 0.6910 (69.10%)

